In [1]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns

Welcome to JupyROOT 6.18/00


### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [2]:
%jsroot on

In [3]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: listOfFile[:] = [x for x in listOfFile if "histograms" not in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/'))

['/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/jetObservables_nanoskim_1.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/jetObservables_nanoskim_2.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/jetObservables_nanoskim_3.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/jetObservables_nanoskim_4.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/jetObservables_nanoskim_5.root', '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/jetObs

In [4]:
nano_directories = [
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_TTTo2L2Nu_new_msoftdrop/210416_002541/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_TTToSemileptonic_new_msoftdrop/210416_125402/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/jetObservables_Skimmer_UL17_TTJets_new_msoftdrop/210416_122322/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-amcatnlo-pythia8/jetObservables_Skimmer_UL17_ST_s-channel_new_msoftdrop/210419_090151/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_top_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_ST_t-channel_top_new_msoftdrop/210419_090045/0000',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_antitop_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_ST_t-channel_antitop_new_msoftdrop/210420_011025/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_ST_tW_top_new_msoftdrop/210420_011129/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17_ST_tW_antitop_new_msoftdrop/210419_085928/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/jetObservables_Skimmer_UL17_WJetsToLNu_new_msoftdrop/210416_003336/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt170to300_new_msoftdrop/210416_003549/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt300to470_new_msoftdrop/210416_004604/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt470to600_new_msoftdrop/210416_003005/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt600to800_new_msoftdrop/210416_003209/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt800to1000_new_msoftdrop/210416_001758/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt1000to1400_new_msoftdrop/210416_003721/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt1400to1800_new_msoftdrop/210416_004736/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt1800to2400_new_msoftdrop/210416_003436/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt2400to3200_new_msoftdrop/210416_004457/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/jetObservables_Skimmer_UL17_QCD_Pt3200toInf_new_msoftdrop/210416_004959/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WW_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_WW_new_msoftdrop/210416_002108/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_WZ_new_msoftdrop/210416_002642/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_new_msoftdrop/210416_004844/0000/'
    ]

In [6]:
dict_MCscaling_UL17 = {
    'TTJets': ["722.8", 285095726103.23967,
               '../TTJets_amcatnloFXFX-pythia8_2017UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 32392787477.908009,
                         '../TTToSemileptonic_powheg_pythia8_2017UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 4759777104.469412, 
                  '../TTTo2L2Nu_powheg_pythia8_2017UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 8110137121728.131836, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 70439807.475972, 
                     '../ST_s-channel_amcatnlo_pythia8_2017UL.root'
                    ],
    #'ST_t-channel_top': ["1.197e+02", 655197632.514000, 
    #                     '../ST_t-channel_top_powheg_pythia8_2017UL.root'
    #                    ],
    'ST_t-channel_antitop': ["7.174e+01", 265124234.844000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 331239851.967700, 
                  '../ST_tW_top_powheg_pythia8_2017UL.root'
                 ],
    'ST_tW_antitop': ["3.251e+01", 303960792.435600, 
                     '../ST_tW_antitop_powheg_pythia8_2017UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Pt_170to300_pythia8_2017UL.root'
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Pt_300to470_pythia8_2017UL.root'   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Pt_470to600_pythia8_2017UL.root'
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Pt_600to800_pythia8_2017UL.root'
                       ],
    #'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
    #                     '../QCD_Pt_800to1000_pythia8_2017UL.root'    
    #                    ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Pt_1000to1400_pythia8_2017UL.root'
                         ],
    
    #'WW': ["7.577e+01", 7876265.259367, 
    #      '../WW_pythia8_2017UL.root'  
    #      ],
    #'WZ': ["1.21e+00", 3970000.000000, 
    #       '../WZ_pythia8_2017UL.root'    
    #      ],
    #'ZZ': ["2.748e+00", 1981800.000000, 
    #       '../ZZ_pythia8_2017UL.root'
    #      ]
    }


In [7]:
lumi=4653.71+9289.53+11758.83
print (lumi)

samples = list(dict_MCscaling_UL17.keys())
dict_items = list(dict_MCscaling_UL17.items())
xsecs = np.array(list(float(dict_items[k][1][0]) for k in np.arange(0,len(dict_items)))) 
nevts = np.array(list(float(dict_items[k][1][1]) for k in np.arange(0,len(dict_items)))) 
fpathsMC = np.array(list((dict_items[k][1][2]) for k in np.arange(0,len(dict_items))))
fpathData = '../SingleMuonAll_2017UL.root'
print (xsecs, samples, nevts, fpathsMC)

25702.07
[7.2280e+02 3.6534e+02 8.8290e+01 5.3680e+04 3.5490e+00 7.1740e+01
 3.2450e+01 3.2510e+01 7.0550e+03 6.1930e+02 5.9240e+01 1.8210e+01
 1.0780e+00] ['TTJets', 'TTToSemiLeptonic', 'TTTo2L2Nu', 'WJetsToLNu', 'ST_s-channel', 'ST_t-channel_antitop', 'ST_tW_top', 'ST_tW_antitop', 'QCD_Pt_170to300', 'QCD_Pt_300to470', 'QCD_Pt_470to600', 'QCD_Pt_600to800', 'QCD_Pt_1000toInf'] [2.85095726e+11 3.23927875e+10 4.75977710e+09 8.11013712e+12
 7.04398075e+07 2.65124235e+08 3.31239852e+08 3.03960792e+08
 3.60276730e+07 4.94796073e+05 5.17383714e+05 1.73188126e+07
 1.46425530e+07] ['../TTJets_amcatnloFXFX-pythia8_2017UL.root'
 '../TTToSemileptonic_powheg_pythia8_2017UL.root'
 '../TTTo2L2Nu_powheg_pythia8_2017UL.root'
 '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root'
 '../ST_s-channel_amcatnlo_pythia8_2017UL.root'
 '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
 '../ST_tW_top_powheg_pythia8_2017UL.root'
 '../ST_tW_antitop_powheg_pythia8_2017UL.root'
 '../QCD_Pt_170to300_pythia8_2017UL.root

In [8]:
import array
from array import array

In [9]:
def Bin(Range):
    bins=np.array([x for x in np.linspace(num=Range[0]+1,start=Range[1],stop=Range[2])])
    return bins

def getRebinnedRescaledHist(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(lumi*xsec/nevt)
    
    return newHist

def getRebinnedRescaledHistfromResp(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    if isMC==True:
        FILE = ROOT.TFile.Open(file_name,'read')
        tree = FILE.Get('jetObservables')
        hist = tree.Get(hist_name)

        hgen = ROOT.TH1F(hist.ProjectionX())
        hreco = ROOT.TH1F(hist.ProjectionY())
        print (hgen, hreco)
        hist.SetDirectory(0)
        ROOT.TH1.AddDirectory(ROOT.kFALSE);

        bins = array('d', np.array(Bin(xrange)))

        newHistreco = ROOT.TH1F(hreco.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
        newHistgen = ROOT.TH1F(hgen.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
        #newHist.SetStats(ROOT.kFALSE)

        newHistreco.Scale(lumi*xsec/nevt)
        newHistgen.Scale(lumi*xsec/nevt)
        return newHistreco, newHistgen 
    else: 
        FILE = ROOT.TFile.Open(file_name,'read')
        tree = FILE.Get('jetObservables')
        hist = tree.Get(hist_name)
        hist.SetDirectory(0)

        ROOT.TH1.AddDirectory(ROOT.kFALSE);

        bins = array('d', np.array(Bin(xrange)))

        newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
        return newHist
    

### W selection 

In [17]:
Hists_WSel=np.array([
    'recoJet_pt_nom_WSel', 'recoJet_mass_nom_WSel', #'recoJetTau21_nom_WSel', 'recoJetTau32_nom_WSel',
    'recoJet_tau_0p5_1_nom_WSel', 'recoJet_tau_1_1_nom_WSel', 'recoJet_tau_2_1_nom_WSel',
    'recoJet_tau_0p5_2_nom_WSel', 'recoJet_tau_1_2_nom_WSel', 'recoJet_tau_2_2_nom_WSel',
    'recoJet_tau_0p5_3_nom_WSel', 'recoJet_tau_1_3_nom_WSel', 'recoJet_tau_2_3_nom_WSel',
    'recoJet_tau_0p5_4_nom_WSel', 'recoJet_tau_1_4_nom_WSel', 'recoJet_tau_2_4_nom_WSel',
    'recoJet_tau21_nom_WSel', 'recoJet_tau32_nom_WSel'])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],#['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/0.01','#tau_{2,1}^{(1)}','#tau_{2,1}^{(1)}'],['Events/0.01','#tau_{3,2}^{(1)}','#tau_{3,2}^{(1)}']])
Range_WSel=[
    [20,0,1000], [11,50,160],#[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1], 
    [20,0,1.],[20,0,1.]
    ]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

16 16 16


In [18]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [19]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.85
y1_l = 0.85

dx_l = 0.25
dy_l = 0.1
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    #####################
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0]))
    TTJets.SetLineColor(ROOT.kCyan-6)
    TTJets.SetFillColor(ROOT.kCyan-6)
    TTJets.SetMarkerColor(ROOT.kCyan-6)
    TTJets.SetMarkerStyle(4)
    TTJets.SetLineWidth(2)
    #####################
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kPink-1)
    TTSemileptonic.SetFillColor(ROOT.kPink-1)
    TTSemileptonic.SetMarkerColor(ROOT.kPink-1)
    TTSemileptonic.SetMarkerStyle(4)
    TTSemileptonic.SetLineWidth(2)
    #####################
    Bkg = ROOT.TH1F("",";;",*Range_WSel[i])
    
    for j in range(2,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        Bkg.Add(hist)
        
    Bkg.SetLineColor(ROOT.kAzure+8)
    Bkg.SetFillColor(ROOT.kAzure+8)
    ######################
    
    TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 1.5*TTSemileptonic.GetMaximum())
    TTSemileptonic.GetYaxis().SetTitle(Histnames_WSel[i][0])
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])   
    TTSemileptonic.Draw("e1")
    c[i].Update()
    TTJets.Draw("same e1")
    
    Data_SingleMuonUL17 = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuonUL17.Add(Bkg.Clone(), -1)
    Data_SingleMuonUL17.SetMarkerSize(1.5)
    Data_SingleMuonUL17.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL17.SetMarkerStyle(20)
    Data_SingleMuonUL17.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL17.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio_TTSemileptonic = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio_TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio_TTSemileptonic.Add(Data_SingleMuonUL17)
    ratio_TTSemileptonic.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio_TTSemileptonic.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio_TTSemileptonic.GetYaxis().SetRangeUser(0.3,1.7)
    ratio_TTSemileptonic.SetLineColor(ROOT.kPink-1)
    ratio_TTSemileptonic.SetMarkerColor(ROOT.kPink-1)
    ratio_TTSemileptonic.SetMarkerStyle(4)

    ratio_TTJets = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio_TTJets.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio_TTJets.Add(Data_SingleMuonUL17)
    ratio_TTJets.SetLineColor(ROOT.kCyan-6)
    ratio_TTJets.SetMarkerColor(ROOT.kCyan-6)
    ratio_TTJets.SetMarkerStyle(4)

    
    TTSemileptonic_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    TTSemileptonic_rt.Add(TTSemileptonic)
    
    TTJets_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    TTJets_rt.Add(TTJets)
    
    ratio_TTSemileptonic.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio_TTSemileptonic.Sumw2()
    ratio_TTSemileptonic.SetStats(0)
    ratio_TTSemileptonic.Divide(TTSemileptonic_rt)
    ratio_TTSemileptonic.GetYaxis().SetLabelSize(0.09)
    ratio_TTSemileptonic.GetXaxis().SetLabelSize(0.09)
    ratio_TTSemileptonic.GetYaxis().SetTitleSize(0.12)
    ratio_TTSemileptonic.GetYaxis().SetTitleOffset(0.4)
    ratio_TTSemileptonic.GetXaxis().SetTitleSize(0.12)
    ratio_TTSemileptonic.GetXaxis().SetTitleOffset(0.85)
    
    ratio_TTJets.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio_TTJets.Sumw2()
    ratio_TTJets.SetStats(0)
    ratio_TTJets.Divide(TTJets_rt)
    ratio_TTJets.GetYaxis().SetLabelSize(0.09)
    ratio_TTJets.GetXaxis().SetLabelSize(0.09)
    ratio_TTJets.GetYaxis().SetTitleSize(0.12)
    ratio_TTJets.GetYaxis().SetTitleOffset(0.4)
    ratio_TTJets.GetXaxis().SetTitleSize(0.12)
    ratio_TTJets.GetXaxis().SetTitleOffset(0.85)
    
    Ratio.append(ratio_TTSemileptonic)
    ratio_TTSemileptonic.Draw("EP")
    ratio_TTJets.Draw("EP same")
    
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} Powheg+Pythia8","lep")
    l[i].AddEntry(TTJets,"t#bar{t} aMC@NLO+Pythia8","lep")
    l[i].AddEntry(Data_SingleMuonUL17,"Data UL17 - Bkg.","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("DataminusBkg/"+Hists_WSel[i]+"_dataminusbkg.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 1000] 25702.07 365.34 32392787477.90801
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_2017UL.root [11, 50, 160] 25702.07 365.34 32392787477.90801
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_WSel ../SingleMuonAll_2017UL.root [30, 0, 0.9] 25702.07 365.34 32392787477.90801
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_WSel ../SingleMuonAll_2017UL.root [35, 0, 0.7] 25702.07 365.34 32392787477.90801
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.6] 25702.07 365.34 32392787477.90801
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.2] 25702.07 365.34 32392787477.90801
#tau_{2}^{(2)}
8 recoJet_tau_0p5_3_nom_WSel ../SingleMuonAll_2017UL.roo

Warning in <TH1F::Add>: Attempt to add histograms with different bin limits
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: png file DataminusBkg/recoJet_pt_nom_WSel_dataminusbkg.png has been created
Warning in <TH1F::Add>: Attempt to add histograms with different bin limits
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: png file DataminusBkg/recoJet_mass_nom_WSel_dataminusbkg.png has been created
Warning in <TH1F::Add>: Attempt to add histograms with different bin limits
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: png file DataminusBkg/recoJet_tau_0p5_1_nom_WSel_dataminusbkg.png has be

### top selection 

In [20]:
Hists_topSel=np.array([
    'recoJet_pt_nom_topSel', 'recoJet_mass_nom_topSel', #'recoJetTau21_nom_topSel', 'recoJetTau32_nom_topSel',
    'recoJet_tau_0p5_1_nom_topSel', 'recoJet_tau_1_1_nom_topSel', 'recoJet_tau_2_1_nom_topSel',
    'recoJet_tau_0p5_2_nom_topSel', 'recoJet_tau_1_2_nom_topSel', 'recoJet_tau_2_2_nom_topSel',
    'recoJet_tau_0p5_3_nom_topSel', 'recoJet_tau_1_3_nom_topSel', 'recoJet_tau_2_3_nom_topSel',
    'recoJet_tau_0p5_4_nom_topSel', 'recoJet_tau_1_4_nom_topSel', 'recoJet_tau_2_4_nom_topSel',
    'recoJet_tau21_nom_topSel', 'recoJet_tau32_nom_topSel',
    ])
#Objects_topSel=np.concatenate((Objects1_topSel,Objects3_topSel),axis=0)

Histnames_topSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],#['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/0.01','#tau_{2,1}^{(1)}','#tau_{2,1}^{(1)}'],['Events/0.01','#tau_{3,2}^{(1)}','#tau_{3,2}^{(1)}'],
    ])
Range_topSel=[
    [30,0,1500], [12,130,250],#[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],
    [20,0,1.],[20,0,1.]
]
print (len(Hists_topSel),len(Histnames_topSel),len(Range_topSel))

16 16 16


In [21]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [22]:
c = np.zeros(len(Range_topSel), ROOT.TCanvas)
l = np.zeros(len(Range_topSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.85
y1_l = 0.85

dx_l = 0.25
dy_l = 0.1
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_topSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_topSel)):
    
    print (i, Hists_topSel[i], fpathData, Range_topSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    #####################
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0]))
    TTJets.SetLineColor(ROOT.kCyan-6)
    TTJets.SetFillColor(ROOT.kCyan-6)
    TTJets.SetMarkerColor(ROOT.kCyan-6)
    TTJets.SetMarkerStyle(4)
    TTJets.SetLineWidth(2)
    #####################
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kPink-1)
    TTSemileptonic.SetFillColor(ROOT.kPink-1)
    TTSemileptonic.SetMarkerColor(ROOT.kPink-1)
    TTSemileptonic.SetMarkerStyle(4)
    TTSemileptonic.SetLineWidth(2)
    #####################
    Bkg = ROOT.TH1F("",";;",*Range_topSel[i])
    
    for j in range(2,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_topSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        Bkg.Add(hist)
        
    Bkg.SetLineColor(ROOT.kAzure+8)
    Bkg.SetFillColor(ROOT.kAzure+8)
    ######################
    
    TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 1.5*TTSemileptonic.GetMaximum())
    TTSemileptonic.GetYaxis().SetTitle(Histnames_topSel[i][0])
    TTSemileptonic.GetXaxis().SetTitle(Histnames_topSel[i][2]) 
    print(Histnames_topSel[i][2])   
    TTSemileptonic.Draw("e1")
    c[i].Update()
    TTJets.Draw("same e1")
    
    Data_SingleMuonUL17 = (getRebinnedRescaledHist(hist_name=Hists_topSel[i], 
                                                file_name=fpathData, xrange=Range_topSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuonUL17.Add(Bkg.Clone(), -1)
    Data_SingleMuonUL17.SetMarkerSize(1.5)
    Data_SingleMuonUL17.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL17.SetMarkerStyle(20)
    Data_SingleMuonUL17.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL17.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio_TTSemileptonic = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio_TTSemileptonic.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio_TTSemileptonic.Add(Data_SingleMuonUL17)
    ratio_TTSemileptonic.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio_TTSemileptonic.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio_TTSemileptonic.GetYaxis().SetRangeUser(0.3,1.7)
    ratio_TTSemileptonic.SetLineColor(ROOT.kPink-1)
    ratio_TTSemileptonic.SetMarkerColor(ROOT.kPink-1)
    ratio_TTSemileptonic.SetMarkerStyle(4)

    ratio_TTJets = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    ratio_TTJets.GetXaxis().SetTitle(Histnames_topSel[i][2])
    ratio_TTJets.Add(Data_SingleMuonUL17)
    ratio_TTJets.SetLineColor(ROOT.kCyan-6)
    ratio_TTJets.SetMarkerColor(ROOT.kCyan-6)
    ratio_TTJets.SetMarkerStyle(4)

    
    TTSemileptonic_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    TTSemileptonic_rt.Add(TTSemileptonic)
    
    TTJets_rt = ROOT.TH1D(Hists_topSel[i],";;",*Range_topSel[i])
    TTJets_rt.Add(TTJets)
    
    ratio_TTSemileptonic.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio_TTSemileptonic.Sumw2()
    ratio_TTSemileptonic.SetStats(0)
    ratio_TTSemileptonic.Divide(TTSemileptonic_rt)
    ratio_TTSemileptonic.GetYaxis().SetLabelSize(0.09)
    ratio_TTSemileptonic.GetXaxis().SetLabelSize(0.09)
    ratio_TTSemileptonic.GetYaxis().SetTitleSize(0.12)
    ratio_TTSemileptonic.GetYaxis().SetTitleOffset(0.4)
    ratio_TTSemileptonic.GetXaxis().SetTitleSize(0.12)
    ratio_TTSemileptonic.GetXaxis().SetTitleOffset(0.85)
    
    ratio_TTJets.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio_TTJets.Sumw2()
    ratio_TTJets.SetStats(0)
    ratio_TTJets.Divide(TTJets_rt)
    ratio_TTJets.GetYaxis().SetLabelSize(0.09)
    ratio_TTJets.GetXaxis().SetLabelSize(0.09)
    ratio_TTJets.GetYaxis().SetTitleSize(0.12)
    ratio_TTJets.GetYaxis().SetTitleOffset(0.4)
    ratio_TTJets.GetXaxis().SetTitleSize(0.12)
    ratio_TTJets.GetXaxis().SetTitleOffset(0.85)
    
    Ratio.append(ratio_TTSemileptonic)
    ratio_TTSemileptonic.Draw("EP")
    ratio_TTJets.Draw("EP same")
    
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} Powheg+Pythia8","lep")
    l[i].AddEntry(TTJets,"t#bar{t} aMC@NLO+Pythia8","lep")
    l[i].AddEntry(Data_SingleMuonUL17,"Data UL17 - Bkg.","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("DataminusBkg/"+Hists_topSel[i]+"_dataminusbkg.png")
    c[i].Close()

0 recoJet_pt_nom_topSel ../SingleMuonAll_2017UL.root [30, 0, 1500] 25702.07 365.34 32392787477.90801
P_{t}/GeV
1 recoJet_mass_nom_topSel ../SingleMuonAll_2017UL.root [12, 130, 250] 25702.07 365.34 32392787477.90801
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_topSel ../SingleMuonAll_2017UL.root [30, 0, 0.9] 25702.07 365.34 32392787477.90801
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_topSel ../SingleMuonAll_2017UL.root [35, 0, 0.7] 25702.07 365.34 32392787477.90801
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.6] 25702.07 365.34 32392787477.90801
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 25702.07 365.34 32392787477.90801
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_topSel ../SingleMuonAll_2017UL.root [20, 0, 0.2] 25702.07 365.34 32392787477.90801
#tau_{2}^{(2)}
8 recoJet_tau_0p5_3_nom_topSel ../Singl

Warning in <TH1F::Add>: Attempt to add histograms with different bin limits
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: png file DataminusBkg/recoJet_pt_nom_topSel_dataminusbkg.png has been created
Warning in <TH1F::Add>: Attempt to add histograms with different bin limits
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: png file DataminusBkg/recoJet_mass_nom_topSel_dataminusbkg.png has been created
Warning in <TH1F::Add>: Attempt to add histograms with different bin limits
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: png file DataminusBkg/recoJet_tau_0p5_1_nom_topSel_dataminusbkg.png 